In [1]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

import pickle

import pandas as pd

import numpy as np

import evaluate

import pyarrow as pa
import pyarrow.dataset as ds
from datasets import Dataset

In [2]:
# Abrindo o objeto dataframe
with open(r'df-pre-processado-cloud2.pickle', 'rb') as pickledfile:
    df = pickle.load(pickledfile)

In [3]:
# Abrindo o objeto dataframe
with open(r'lista-splits.pickle', 'rb') as pickledfile:
    lista_splits = pickle.load(pickledfile)

In [4]:
metric = evaluate.load("accuracy")

In [5]:
def compute_metrics(eval_pred):

    logits, labels = eval_pred

    predictions = np.argmax(logits, axis=-1)

    return metric.compute(predictions=predictions, references=labels)

In [6]:
bertmodel = AutoModelForSequenceClassification.from_pretrained('neuralmind/bert-large-portuguese-cased', num_labels = 5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-large-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
training_args = TrainingArguments(output_dir="treino/split1", eval_strategy="epoch", use_cpu=False, no_cuda=False)

t_ = lista_splits[0][1]
v_ = lista_splits[0][2]

df_teste = pd.DataFrame.from_dict(
    {'input_ids': df.loc[t_, 'input_ids'], 'label': df.loc[t_, 'label'], 'attention_mask':df.loc[t_, 'attention_mask'] }
)
hg_dataset_teste = Dataset(pa.Table.from_pandas(df_teste))

df_val = pd.DataFrame.from_dict(
    {'input_ids': df.loc[v_, 'input_ids'], 'label': df.loc[v_, 'label'], 'attention_mask':df.loc[v_, 'attention_mask'] }
)
hg_dataset_val = Dataset(pa.Table.from_pandas(df_val))

trainer = Trainer(
    model=bertmodel,
    args=training_args,
    train_dataset=hg_dataset_teste,
    eval_dataset=hg_dataset_val,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
